# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 20 2022 12:12PM,239956,20,8168096,"Exact-Rx, Inc.",Released
1,Jun 20 2022 12:12PM,239956,20,8169329,"Exact-Rx, Inc.",Released
2,Jun 20 2022 12:12PM,239956,20,8169330,"Exact-Rx, Inc.",Released
3,Jun 20 2022 12:12PM,239956,20,8169331,"Exact-Rx, Inc.",Released
4,Jun 20 2022 12:12PM,239956,20,8169332,"Exact-Rx, Inc.",Released
5,Jun 20 2022 12:12PM,239956,20,8169333,"Exact-Rx, Inc.",Released
6,Jun 20 2022 12:12PM,239956,20,8169334,"Exact-Rx, Inc.",Released
7,Jun 20 2022 12:12PM,239956,20,8169335,"Exact-Rx, Inc.",Released
8,Jun 20 2022 12:12PM,239956,20,8169336,"Exact-Rx, Inc.",Released
9,Jun 20 2022 12:12PM,239956,20,8169337,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,239952,Released,23
39,239953,Released,29
40,239955,Released,52
41,239956,Released,22
42,239957,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
239952,NaN,23.0
239953,NaN,29.0
239955,NaN,52.0
239956,NaN,22.0
239957,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
239878,0.0,1.0
239892,0.0,1.0
239896,0.0,1.0
239897,0.0,1.0
239898,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
239878,0,1
239892,0,1
239896,0,1
239897,0,1
239898,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,239878,0,1
1,239892,0,1
2,239896,0,1
3,239897,0,1
4,239898,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,239878,,1
1,239892,,1
2,239896,,1
3,239897,,1
4,239898,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc."
22,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation
74,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC"
75,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc."
104,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC"
127,Jun 20 2022 11:20AM,239949,10,Bio-PRF
155,Jun 20 2022 11:18AM,239950,10,Emerginnova
156,Jun 20 2022 11:16AM,239946,18,"ClearSpec, LLC"
157,Jun 20 2022 11:14AM,239948,10,"Methapharm, Inc."
158,Jun 20 2022 11:14AM,239947,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",,22
1,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,,52
2,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC",,1
3,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc.",,29
4,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC",,23
5,Jun 20 2022 11:20AM,239949,10,Bio-PRF,,28
6,Jun 20 2022 11:18AM,239950,10,Emerginnova,,1
7,Jun 20 2022 11:16AM,239946,18,"ClearSpec, LLC",,1
8,Jun 20 2022 11:14AM,239948,10,"Methapharm, Inc.",,1
9,Jun 20 2022 11:14AM,239947,10,"Senseonics, Incorporated",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22,
1,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52,
2,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC",1,
3,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc.",29,
4,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC",23,
5,Jun 20 2022 11:20AM,239949,10,Bio-PRF,28,
6,Jun 20 2022 11:18AM,239950,10,Emerginnova,1,
7,Jun 20 2022 11:16AM,239946,18,"ClearSpec, LLC",1,
8,Jun 20 2022 11:14AM,239948,10,"Methapharm, Inc.",1,
9,Jun 20 2022 11:14AM,239947,10,"Senseonics, Incorporated",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22,
1,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52,
2,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC",1,
3,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc.",29,
4,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC",23,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22.0,NaN
1,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52.0,NaN
2,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC",1.0,NaN
3,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc.",29.0,NaN
4,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC",23.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 12:12PM,239956,20,"Exact-Rx, Inc.",22.0,0.0
1,Jun 20 2022 12:12PM,239955,10,ISDIN Corporation,52.0,0.0
2,Jun 20 2022 12:10PM,239957,12,"SD Head USA, LLC",1.0,0.0
3,Jun 20 2022 12:01PM,239953,20,"ACI Healthcare USA, Inc.",29.0,0.0
4,Jun 20 2022 11:58AM,239952,19,"Innogenix, LLC",23.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4078822,262.0,19.0
12,239957,1.0,0.0
15,719771,39.0,39.0
16,1199512,5.0,0.0
18,479844,2.0,0.0
19,719794,24.0,1.0
20,719808,66.0,0.0
21,479856,2.0,0.0
22,959739,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4078822,262.0,19.0
1,12,239957,1.0,0.0
2,15,719771,39.0,39.0
3,16,1199512,5.0,0.0
4,18,479844,2.0,0.0
5,19,719794,24.0,1.0
6,20,719808,66.0,0.0
7,21,479856,2.0,0.0
8,22,959739,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,262.0,19.0
1,12,1.0,0.0
2,15,39.0,39.0
3,16,5.0,0.0
4,18,2.0,0.0
5,19,24.0,1.0
6,20,66.0,0.0
7,21,2.0,0.0
8,22,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,262.0
1,12,Released,1.0
2,15,Released,39.0
3,16,Released,5.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Executing,19.0,0.0,39.0,0.0,0.0,1.0,0.0,0.0,0.0
Released,262.0,1.0,39.0,5.0,2.0,24.0,66.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Executing,19.0,0.0,39.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Released,262.0,1.0,39.0,5.0,2.0,24.0,66.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Executing,19.0,0.0,39.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Released,262.0,1.0,39.0,5.0,2.0,24.0,66.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()